# Verification of Portfolio Calculations
Independent verification of contributed capital, FX P&L, and cash balance

In [ ]:
import sys
sys.path.insert(0, '../src')

from data_loader import load_transactions, load_positions
from metrics import calculate_portfolio_metrics
import polars as pl

In [ ]:
# Load data
tx = load_transactions().sort('date')
pos = load_positions()
hist = calculate_portfolio_metrics(tx)
final = hist.row(-1, named=True)

print(f"Loaded {tx.height} transactions")
print(f"Portfolio history: {hist.height} days")

## 1. Verify JPY Deposits and FX Conversions

In [ ]:
# Get all JPY deposits
deposits = tx.filter((pl.col('type') == 'Deposit') & (pl.col('currency') == 'JPY'))
total_jpy_deposited = deposits['amount'].sum()

print("JPY DEPOSITS:")
print("=" * 80)
for r in deposits.iter_rows(named=True):
    print(f"{r['date']}: ¥{r['amount']:,.0f}")
print("-" * 80)
print(f"Total JPY deposited: ¥{total_jpy_deposited:,.0f}")
print()

In [ ]:
# Get all FX transactions
fx = tx.filter(pl.col('type') == 'FX')
total_usd_from_fx = fx['quantity'].sum()
total_jpy_for_fx = fx['amount'].sum()

print("FX CONVERSIONS:")
print("=" * 80)
print(f"Total USD received from FX: ${total_usd_from_fx:,.2f}")
print(f"Total JPY spent on FX: ¥{abs(total_jpy_for_fx):,.2f}")
print(f"JPY remaining: ¥{total_jpy_deposited + total_jpy_for_fx:,.2f}")
print(f"Average FX rate: ¥{abs(total_jpy_for_fx) / total_usd_from_fx:.2f}/USD")
print()

## 2. Calculate Contributed Capital (Manual Verification)

In [ ]:
# Manually trace through transactions to calculate contributed capital
# This values JPY deposits at the FX rate at the time of deposit

contributed_manual = 0.0
current_fx_rate = 143.28  # First FX rate

print("CONTRIBUTED CAPITAL CALCULATION:")
print("=" * 80)
print(f"{'Date':<12} {'Type':<10} {'Amount':<15} {'FX Rate':<10} {'USD Value':<15}")
print("-" * 80)

for r in tx.iter_rows(named=True):
    # Update current FX rate
    if r['type'] == 'FX' and r['quantity'] != 0:
        current_fx_rate = abs(r['amount']) / abs(r['quantity'])
    
    # Process deposits
    if r['type'] == 'Deposit':
        if r['currency'] == 'JPY':
            usd_value = r['amount'] / current_fx_rate
            contributed_manual += usd_value
            print(f"{str(r['date']):<12} {'JPY Dep':<10} ¥{r['amount']:>12,.0f} {current_fx_rate:>9.2f} ${usd_value:>13,.2f}")
        else:
            contributed_manual += r['amount']
            print(f"{str(r['date']):<12} {'USD Dep':<10} ${r['amount']:>12,.2f} {'-':>9} ${r['amount']:>13,.2f}")

print("-" * 80)
print(f"{'TOTAL':<12} {'':<10} {'':<15} {'':<10} ${contributed_manual:>13,.2f}")
print()
print(f"Manual calculation: ${contributed_manual:,.2f}")
print(f"From metrics.py: ${final['contributed']:,.2f}")
print(f"Difference: ${abs(contributed_manual - final['contributed']):,.2f}")
print()

## 3. Calculate FX P&L (Manual Verification)

In [ ]:
# FX P&L = (Actual USD received from FX + unconverted JPY in USD) - (JPY deposits valued at deposit time)

jpy_remaining = total_jpy_deposited + total_jpy_for_fx
jpy_remaining_usd = jpy_remaining / current_fx_rate

fx_pnl_manual = total_usd_from_fx + jpy_remaining_usd - contributed_manual

print("FX P&L CALCULATION:")
print("=" * 80)
print(f"Actual USD received from FX conversions: ${total_usd_from_fx:,.2f}")
print(f"Unconverted JPY (¥{jpy_remaining:,.2f}) at current rate: ${jpy_remaining_usd:,.2f}")
print(f"Total actual USD value: ${total_usd_from_fx + jpy_remaining_usd:,.2f}")
print()
print(f"JPY deposits valued at deposit time: ${contributed_manual:,.2f}")
print()
print(f"FX P&L (manual): ${fx_pnl_manual:,.2f}")
print(f"FX P&L from metrics.py: ${final['fx_pnl']:,.2f}")
print(f"Difference: ${abs(fx_pnl_manual - final['fx_pnl']):,.2f}")
print()

## 4. Verify Investment Flows

In [ ]:
# Calculate all investment income and expenses

# Stock trades
buys = tx.filter((pl.col('type') == 'Trade') & (pl.col('quantity') > 0))
sells = tx.filter((pl.col('type') == 'Trade') & (pl.col('quantity') < 0))

total_buy_cost = sum([abs(r['amount']) + abs(r['commission']) for r in buys.iter_rows(named=True)])
total_sell_proceeds = sum([r['amount'] - r['commission'] for r in sells.iter_rows(named=True)])

# Dividends and costs
dividends = tx.filter(pl.col('type') == 'Dividend')['amount'].sum()
taxes = abs(tx.filter(pl.col('type') == 'Tax')['amount'].sum())
fees = abs(tx.filter(pl.col('type') == 'Fee')['amount'].sum())

print("INVESTMENT FLOWS:")
print("=" * 80)
print(f"Total spent on stock purchases: ${total_buy_cost:,.2f}")
print(f"Total received from stock sales: ${total_sell_proceeds:,.2f}")
print(f"Dividends received: ${dividends:,.2f}")
print(f"Taxes paid: ${taxes:,.2f}")
print(f"Fees paid: ${fees:,.2f}")
print()

# Realized P&L from metrics
print(f"Realized capital gains (from metrics.py): ${final['realized_pnl']:,.2f}")
print(f"Net dividends (from metrics.py): ${dividends - taxes:,.2f}")
print()

## 5. Verify Cash Balance Formula

In [ ]:
# Cash = Contributed - Invested + Realized PNL + Dividends + FX PNL - Taxes - Fees
# (Note: commissions are already included in invested capital for buys and in realized PNL for sells)

expected_cash = (
    final['contributed'] 
    - final['invested_capital'] 
    + final['realized_pnl'] 
    + final['dividends'] 
    + final['fx_pnl']
    - taxes 
    - fees
)

print("CASH BALANCE VERIFICATION:")
print("=" * 80)
print(f"Contributed capital:          ${final['contributed']:>12,.2f}")
print(f"Invested capital:           - ${final['invested_capital']:>12,.2f}")
print(f"Realized capital gains:     + ${final['realized_pnl']:>12,.2f}")
print(f"Dividends (gross):          + ${final['dividends']:>12,.2f}")
print(f"FX P&L:                     + ${final['fx_pnl']:>12,.2f}")
print(f"Taxes:                      - ${taxes:>12,.2f}")
print(f"Fees:                       - ${fees:>12,.2f}")
print("-" * 80)
print(f"Expected cash:                ${expected_cash:>12,.2f}")
print(f"Actual cash:                  ${final['cash']:>12,.2f}")
print(f"Difference:                   ${abs(expected_cash - final['cash']):>12,.2f}")
print()

if abs(expected_cash - final['cash']) < 0.01:
    print("✓ Cash balance formula is CORRECT!")
else:
    print("✗ Cash balance has discrepancy")
print()

## 6. Answer: Why is Invested Capital close to Contributed?

In [ ]:
print("WHY IS INVESTED CAPITAL CLOSE TO CONTRIBUTED CAPITAL?")
print("=" * 80)
print()
print(f"Contributed capital: ${final['contributed']:,.2f}")
print(f"Invested capital:    ${final['invested_capital']:,.2f}")
print(f"Difference:          ${final['invested_capital'] - final['contributed']:,.2f}")
print()
print("Breakdown of what happened to your contributed capital:")
print()
print("GAINS:")
print(f"  + Realized capital gains: ${final['realized_pnl']:,.2f}")
print(f"  + Net dividends:          ${final['dividends'] - taxes:,.2f}")
print(f"  Total gains:              ${final['realized_pnl'] + final['dividends'] - taxes:,.2f}")
print()
print("LOSSES:")
print(f"  - FX conversion loss:     ${abs(final['fx_pnl']):,.2f}")
print(f"  - Fees:                   ${fees:,.2f}")
print(f"  Total losses:             ${abs(final['fx_pnl']) + fees:,.2f}")
print()
net_realized = final['realized_pnl'] + final['dividends'] - taxes + final['fx_pnl'] - fees
print(f"Net realized income:        ${net_realized:,.2f}")
print(f"Less: Cash not invested:    ${final['cash']:,.2f}")
print(f"Added to investments:       ${net_realized - final['cash']:,.2f}")
print()
print(f"Verification: ${final['contributed']:,.2f} + ${net_realized - final['cash']:,.2f} = ${final['contributed'] + net_realized - final['cash']:,.2f}")
print(f"Invested capital: ${final['invested_capital']:,.2f}")
print(f"Match: {abs((final['contributed'] + net_realized - final['cash']) - final['invested_capital']) < 0.01}")
print()

## 7. Summary

In [ ]:
print("SUMMARY")
print("=" * 80)
print()
print(f"Your invested capital (${final['invested_capital']:,.2f}) is only")
print(f"${final['invested_capital'] - final['contributed']:,.2f} more than your contributed capital")
print(f"(${final['contributed']:,.2f}) because:")
print()
print(f"1. You earned ${final['realized_pnl'] + final['dividends'] - taxes:,.2f} from investments")
print(f"   (${final['realized_pnl']:,.2f} capital gains + ${final['dividends'] - taxes:,.2f} net dividends)")
print()
print(f"2. But you lost ${abs(final['fx_pnl']):,.2f} from currency conversion")
print(f"   (JPY weakened from ~143 to ~148 yen/dollar on average)")
print()
print(f"3. And paid ${fees:,.2f} in fees")
print()
print(f"4. Plus ${final['cash']:,.2f} is sitting in cash (uninvested)")
print()
print(f"Net effect: ${final['realized_pnl'] + final['dividends'] - taxes:,.2f} - ${abs(final['fx_pnl']):,.2f} - ${fees:,.2f} - ${final['cash']:,.2f}")
print(f"          = ${final['realized_pnl'] + final['dividends'] - taxes + final['fx_pnl'] - fees - final['cash']:,.2f}")
print()